Data design:
The dataset we pulled today contains a large number of duplicate entries. After deduplication, only 94 unique researchers remain. So I delete this file and we could refer to the data_design_withoutdate for now. 
This code is for basic data design and cleaning. The objective is to generate three tables from the raw full dataset-JSON data. 

We plan to generate three tables from the raw JSON data:

1. Researcher table: researcher_df 
This table contains one row per researcher.
Required fields include: 
"researcher_id": a unique identifier for each researcher; 
"first_name"; 
"last_name"; 
"email"; 
"title"; 
"overview":
"department_raw" (parsed from the title directly before cleaning); 
"role": the role of this research in Duke University(for example, professor, assistant professor, graduate student, etc.)The role field for each researcher is determined through a two-stage extraction pipeline. The general principle is: use the primaryAppointment.title first; if that fails, fall back to parsing the overview text. All matching is case-insensitive unless otherwise specified.
pub_count: the overall amounts of publication of this researcher;
latest_pub_date: the newest publication date.


2. Publication table: publication_clean_df 
****** There are a total of 481,572 author–publication rows, but the number of orginal pub_id(pulled out directly from duke database)'s  values is also 481,572. This means that each pub_id appears only once, and there is no case where two authors share the same pub_id. Therefore, pub_id cannot be used as an indicator of co-authorship, because it does not represent shared publications among multiple researchers. 

****** In this case, use "title to create unique_ids and coauthor pairs. (After testing, there is no missing in titles)

this table contains one row per publication.
Required fields include:

"pub_unique_id": the unique_id we created by titles. 
"researcher_id": researcher_id,
"researcher_full_name": first_name,
"pub_title": pub_title,
"pub_abstract": pub_abstract,
"pub_doi": pub_doi,
"duke_authors": All Duke-affiliated authors matched from the Duke researcher database.
"raw_pub_ids": All original pub_id values associated with this publication in the Duke source data.
"all_authors": all_authors, including people who are not matched via Duke database,
"pub_date": publicated date of this publication/paper.


3. Coauthor relationship: coauthor_pairs_final  
This table contains one row per co-author pair. 
Based on the unique_ids we used "title" to create, we created the deduplicated author.
Required fields include:
Researcher_A_id,
Researcher_B_id,
Researcher_A_name,
Researcher_B_name,
joint_pub_numbers: how many publications they collaborated. 



In [2]:
# packages
import pandas as pd
import numpy as np
import json
import requests
import os
!pip install nanoid
from nanoid import generate

In [17]:
# import data
import json

with open("people_2025-11-24_11-13-33.json", "r") as f:
    data = json.load(f)


In [18]:
# data strcuture
def print_json_structure(d, indent=0):
    prefix = "  " * indent
    if isinstance(d, dict):
        for k, v in d.items():
            print(f"{prefix}{k}: ({type(v).__name__})")
            print_json_structure(v, indent + 1)
    elif isinstance(d, list):
        print(f"{prefix}[list of length {len(d)}]")
        if len(d) > 0:
            print_json_structure(d[0], indent + 1)
    else:
        # primitive value -> 不展开
        print(f"{prefix}{d} ({type(d).__name__})")

print_json_structure(data)


[list of length 11600]
  currentResearch: (list)
    [list of length 1]
      id: (str)
        38405 (str)
  email: (str)
    aaliya.aaliya@duke.edu (str)
  firstName: (str)
    Aaliya (str)
  lastName: (str)
    Aaliya (str)
  overview: (str)
    Aaliya is a Ph.D. Candidate working under the supervision of Prof. Dr. Brian McAdoo in PlanetLab in the Division of Earth and Climate Sciences at Duke University&rsquo;s Nicholas School of the Environment. Her doctoral research focuses on understanding the planetary-scale changes such as climate change impacting the health and well-being of rural mountain communities in the Hindu Kush Himalayan (HKH) Region, focusing specifically on Nepal and Pakistan. (str)
  primaryAppointment: (NoneType)
    None (NoneType)
  publications: (dict)
    count: (int)
      0 (int)
    results: (list)
      [list of length 0]


In [19]:
# need to figure out what the 'currentResearch' refers to.
for i in range(10):
    print(data[i]['currentResearch'])

[{'id': '38405'}]
[]
[]
[]
[]
[]
[{'id': '11060'}]
[]
[]
[]


In [20]:
# generate unique researcher id. 
from nanoid import generate  # make sure nanoid is installed: pip install nanoid

# data is already a list of researchers
print(type(data), len(data))  # just to confirm

for person in data:
    person["unique_id"] = generate(size=12)  # e.g. "a8Gs93KdL0pQ"


<class 'list'> 11600


In [21]:
# role extraction from "PrimaryAppionment[title]

import re

def get_role_phrase(raw_title: str):
    """Extract the role phrase before 'in/of/at' or comma."""
    if not raw_title:
        return None
    t = raw_title.strip()
    m = re.match(r'^(.*?)(?:\s+(?:in|of|at)\b|,)', t, flags=re.I)
    if m:
        return m.group(1).strip()
    return t


def normalize_role(raw_title: str):
    """Normalize academic role titles into a standard label."""
    if not raw_title:
        return None
    
    # Step 1: extract core role phrase
    t = get_role_phrase(raw_title)
    if not t:
        return None
    s = t.strip()

    # --- Matching rules in priority order ---

    # Emeritus/Emerita
    if re.search(r'\bProfessor\b.*\bEmerit', s, flags=re.I):
        return 'Professor Emeritus'

    # Professor of the Practice
    if re.search(r'\bProfessor\s+of\s+the\s+Practice\b', s, flags=re.I):
        return 'Professor of the Practice'
    if re.search(r'\bAssociate\s+Professor\s+of\s+the\s+Practice\b', s, flags=re.I):
        return 'Associate Professor of the Practice'
    if re.search(r'\bAssistant\s+Professor\s+of\s+the\s+Practice\b', s, flags=re.I):
        return 'Assistant Professor of the Practice'

    # Special prefixes
    if re.search(r'\bAdjunct\s+Professor\b', s, flags=re.I):
        return 'Adjunct Professor'
    if re.search(r'\bClinical\s+Professor\b', s, flags=re.I):
        return 'Clinical Professor'
    if re.search(r'\bResearch\s+Professor\b', s, flags=re.I):
        return 'Research Professor'
    if re.search(r'\bVisiting\s+Professor\b', s, flags=re.I):
        return 'Visiting Professor'
    
    # Medical titles
    if re.search(r'\bMedical\s+Assistant\s+Professor\b', s, flags=re.I):
        return 'Medical Assistant Professor'
    if re.search(r'\bMedical\s+Associate\s+Professor\b', s, flags=re.I):
        return 'Medical Associate Professor'
    if re.search(r'\bMedical\s+Professor\b', s, flags=re.I):
        return 'Medical Professor'

    # Standard professor ranks
    if re.search(r'\bAssociate\s+Professor\b', s, flags=re.I):
        return 'Associate Professor'
    if re.search(r'\bAssistant\s+Professor\b', s, flags=re.I):
        return 'Assistant Professor'
    if re.search(r'\bProfessor\b', s, flags=re.I):
        return 'Professor'

    # Clinical roles
    if re.search(r'\bClinical\s+Faculty\b', s, flags=re.I):
        return 'Clinical Faculty'
    if re.search(r'\bAssociate\s+Clinical\s+Professor\b', s, flags=re.I):
        return 'Associate Clinical Professor'
    if re.search(r'\bAssistant\s+Clinical\s+Professor\b', s, flags=re.I):
        return 'Assistant Clinical Professor'
    if re.search(r'\bClinical\s+Associate\b', s, flags=re.I):
        return 'Clinical Associate'
    if re.search(r'\bClinical\s+Affiliate\b', s, flags=re.I):
        return 'Clinical Affiliate'

    # Lecturer / Fellow
    if re.search(r'\bSenior\s+Lecturing\s+Fellow\b', s, flags=re.I):
        return 'Senior Lecturing Fellow'
    if re.search(r'\bLecturing\s+Fellow\b', s, flags=re.I):
        return 'Lecturing Fellow'
    if re.search(r'\bSenior\s+Lecturer\b', s, flags=re.I):
        return 'Senior Lecturer'
    if re.search(r'\bLecturer\b', s, flags=re.I):
        return 'Lecturer'

    # Instructor family
    if re.search(r'\bMedical\s+Instructor\b', s, flags=re.I):
        return 'Medical Instructor'
    if re.search(r'\bInstructor\b', s, flags=re.I):
        return 'Instructor'

    # In Residence / Artist in Residence / Scholar in Residence
    if re.search(r'\bExecutive\s+In\s+Residence\b', s, flags=re.I):
        return 'Executive in Residence'
    if re.search(r'\bArtist\s+In\s+Residence\b', s, flags=re.I):
        return 'Artist in Residence'
    if re.search(r'\bScholar\s+In\s+Residence\b', s, flags=re.I):
        return 'Scholar in Residence'

    # Fallback: return original phrase
    return s


In [22]:
# Step 1: extract title from primaryAppointment
def extract_pa_title(pa):
    if isinstance(pa, dict):
        return pa.get("title")
    return None

# Step 2: add the new field to each researcher dict
for person in data:   # data is your full JSON list
    pa_title = extract_pa_title(person.get("primaryAppointment"))
    role = normalize_role(pa_title)  # use the function we defined earlier
    person["role"] = role


In [23]:
# if the role is null, continue to fill the role with "overview"

import re

# 1. Regex pattern for roles in overview (case-insensitive)
role_pattern_overview = re.compile(
    r"("
    r"ph\.?d\.?\s*candidate|"        # PhD Candidate / Ph.D. Candidate / phd candidate
    r"doctoral\s+student|"           # Doctoral Student
    r"graduate\s+student|"           # Graduate Student
    r"\bdirector\b|"                 # Director
    r"\bstudent\b"                   # Student (fallback; used only if nothing else matched)
    r")",
    re.IGNORECASE
)

# 2. Fill role from overview ONLY when existing role is None
filled_from_overview = 0

for person in data:
    # Only try overview if role is currently None
    if person.get("role") is None:
        overview = person.get("overview") or ""
        
        # Search in overview text
        match = role_pattern_overview.search(overview)
        if match:
            # Use the matched string as role (keep original casing from overview)
            person["role"] = match.group(0).strip()
            filled_from_overview += 1

print("Number of roles filled from overview:", filled_from_overview)


Number of roles filled from overview: 348


In [24]:
# department/major 
import re

def extract_department_from_title(primary_appointment):
    """
    primary_appointment is either:
      - None
      - dict like {"title": "Assistant Professor in ..."}
    """
    # 1. If no primaryAppointment
    if primary_appointment is None:
        return None
    
    # 2. Extract string from {"title": "..."} 
    if isinstance(primary_appointment, dict):
        title = primary_appointment.get("title", None)
    else:
        # fallback: if it's already a string
        title = primary_appointment
    
    if not isinstance(title, str):
        return None
    
    t = title.strip()
    if not t:
        return None

    lower = t.lower()

    # --- extraction rules ---
    idx_in = lower.rfind(" in ")
    if idx_in != -1:
        dept = t[idx_in + len(" in "):].strip()
    else:
        idx_of = lower.rfind(" of ")
        if idx_of != -1:
            dept = t[idx_of + len(" of "):].strip()
        else:
            return None

    # remove "the "
    dept = re.sub(r"^\s*the\s+", "", dept, flags=re.IGNORECASE)

    # remove prefixes
    dept = re.sub(r"^(Department|Division|School|Program|Centre|Center)\s+of\s+", "", dept, flags=re.IGNORECASE).strip()

    # Handle "Department of X" inside string
    m = re.search(r"Department\s+of\s+(.+)", dept, flags=re.IGNORECASE)
    if m:
        dept = m.group(1).strip()

    # Handle "School of Nursing" → "Nursing"
    m2 = re.search(r"School\s+of\s+(.+)", dept, flags=re.IGNORECASE)
    if m2:
        dept = m2.group(1).strip()

    return dept if dept else None


# --- Attach to your data ---
for person in data:
    person["department_raw"] = extract_department_from_title(person.get("primaryAppointment"))


In [25]:

# generate table one: researcher table 
import pandas as pd

rows = []

for person in data:   # data is your list of 11547 researchers

    # 1. unique researcher id
    rid = person.get("unique_id")

    # 2. name fields
    first = person.get("firstName")
    last = person.get("lastName")
    full = f"{first} {last}".strip() if first or last else None

    # 3. email
    email = person.get("email")

    # 4. primary appointment title
    pa = person.get("primaryAppointment")
    if isinstance(pa, dict):
        title = pa.get("title")
    else:
        title = None

    # 5. department_raw from your earlier extraction
    dept = person.get("department_raw")

    # 6. role (you already extracted earlier)
    role = person.get("role")

    # 7. publication count
    pubs = person.get("publications", {})
    pub_count = pubs.get("count", 0) if isinstance(pubs, dict) else 0

    # 8. latest publication date (take max over all publicationDate.date)
    latest_pub = None
    if isinstance(pubs, dict):
        results = pubs.get("results", [])
        dates = []
        for p in results:
            pub_info = p.get("publication", {}) or {}
            pub_date_obj = pub_info.get("publicationDate") or {}
            date_str = pub_date_obj.get("date")   # e.g. "2025-02-01"
            if date_str:
                dates.append(date_str)
        if dates:
            latest_pub = max(dates)

    #9. overview
    overview = person.get("overview")

    # add row
    rows.append({
        "researcher_id": rid,
        "first_name": first,
        "last_name": last,
        "full_name": full,
        "email": email,
        "title": title,
        "department_raw": dept,
        "role": role,
        "overview": overview,
        "pub_count": pub_count,
        "latest_pub_date": latest_pub,
    
    })

# convert to DataFrame
researcher_df = pd.DataFrame(rows)

researcher_df.head()


,researcher_id,first_name,last_name,full_name,email,title,department_raw,role,overview,pub_count,latest_pub_date
0,O15-jz9TO8rK,Aaliya,Aaliya,Aaliya Aaliya,aaliya.aaliya@duke.edu,None,None,Ph.D. Candidate,Aaliya is a Ph.D. Candidate working under the ...,0,None
1,YUtqevyNtls3,Ehsan,Abadi,Ehsan Abadi,ehsan.abadi@duke.edu,Associate Professor in Radiology,Radiology,Associate Professor,"<p><strong>Ehsan Abadi, PhD</strong> is an ima...",168,2025-11-01
2,5aP0_VOFwhR4,Bijan,Abar,Bijan Abar,bijan.abar@duke.edu,None,None,None,None,1,2025-02-01
3,C_0nRjiy4Eeq,Aamna,Abbasi,Aamna Abbasi,aamna.abbasi@duke.edu,None,None,None,None,0,None
4,hoxMQY2SKrOS,James,Abbruzzese,James Abbruzzese,james.abbruzzese@duke.edu,Professor Emeritus of Medicine,Medicine,Professor Emeritus,My research interests include the clinical stu...,585,2025-02-01


In [26]:
#generate table two: publication table 
# step one - original publication-author 
import pandas as pd

pub_rows = []

for person in data:  # data is your list of researchers
    
    # researcher basic info
    researcher_id = person.get("unique_id")
    first = person.get("firstName")
    last = person.get("lastName")
    full_name = f"{first} {last}".strip() if first or last else None
    
    # publications block
    pubs = person.get("publications", {})
    if isinstance(pubs, dict):
        results = pubs.get("results", [])
    else:
        results = []
    
    # loop over each publication for this researcher
    for p in results:
        raw_pub_id = p.get("id")
        pub_info = p.get("publication", {}) or {}
        
        pub_title = pub_info.get("title")
        pub_abstract = pub_info.get("abstract")
        pub_doi = pub_info.get("doi")
        
        all_authors_obj = pub_info.get("allAuthors", {}) or {}
        all_authors = all_authors_obj.get("fullList")

        # NEW: extract publicationDate
        # structure is like:  "publicationDate": {"date": "2025-02-01"}
        date_obj = pub_info.get("publicationDate", {}) or {}
        pub_date = date_obj.get("date")   # may be None

        
        pub_rows.append({
            "raw_pub_id": raw_pub_id,
            "researcher_id": researcher_id,
            "full_name": full_name,
            "pub_title": pub_title,
            "pub_abstract": pub_abstract,
            "pub_doi": pub_doi,
            "all_authors": all_authors,
            "pub_date":pub_date
        })

# build the publication DataFrame
raw_pub_df = pd.DataFrame(pub_rows)

# 快速看一下前几行
raw_pub_df.head()


,raw_pub_id,researcher_id,full_name,pub_title,pub_abstract,pub_doi,all_authors,pub_date
0,0682983-1692689-8,YUtqevyNtls3,Ehsan Abadi,Evaluation of unified harmonization of CT imag...,"BACKGROUND: In medical imaging, harmonization ...",10.1002/mp.70084,"Xia, S-J; Vancoillie, L; Sotoudeh-Paima, S; Za...",2025-11-01
1,0682983-1689232-8,YUtqevyNtls3,Ehsan Abadi,Comparison of quantitative lung measures in lo...,Photon-counting detector (PCD) computed tomogr...,10.1088/2057-1976/ae0e27,"AlArab, N; McIntosh, MJ; Guo, J; Atha, J; Kizh...",2025-10-30
2,0682983-1690366-8,YUtqevyNtls3,Ehsan Abadi,An end-to-end CT simulation framework with gra...,BACKGROUND: Virtual imaging trials (VIT) facil...,10.1002/mp.70066,"Kavuri, A; Fryling, M; Felice, N; Malvin, L; C...",2025-10-01
3,0682983-1686720-8,YUtqevyNtls3,Ehsan Abadi,Liver fat quantification using deep silicon ph...,BACKGROUND: Accurate liver fat quantification ...,10.1093/radadv/umaf031,"Panta, RK; Yin, Z; Grönberg, F; Wildman-Tobrin...",2025-09-01
4,0682983-1683612-8,YUtqevyNtls3,Ehsan Abadi,Development of a deep learning based approach ...,Conventional approaches to material decomposit...,10.1038/s41598-025-09739-9,"Rajagopal, JR; Rapaka, S; Farhadi, F; Abadi, E...",2025-08-06


In [27]:
#generate table two: publication table 
import pandas as pd
from nanoid import generate


clean_pub = raw_pub_df.dropna(subset=["pub_title"]).copy()

grouped = clean_pub.groupby("pub_title")

new_pub_rows = []

for title, group in grouped:

    pub_unique_id = generate(size=12)

    duke_authors = "; ".join(group["full_name"].dropna().unique())

    raw_pub_ids = list(group["raw_pub_id"].dropna().unique())

    pub_abstract = group["pub_abstract"].iloc[0]
    pub_doi = group["pub_doi"].iloc[0]
    pub_date = group["pub_date"].iloc[0]

    new_pub_rows.append({
        "pub_unique_id": pub_unique_id,
        "pub_title": title,
        "duke_authors": duke_authors,
        "raw_pub_ids": raw_pub_ids,          
        "pub_abstract": pub_abstract,
        "pub_doi": pub_doi,
        "pub_date": pub_date
    })

publication_clean_df = pd.DataFrame(new_pub_rows)

publication_clean_df.head()



,pub_unique_id,pub_title,duke_authors,raw_pub_ids,pub_abstract,pub_doi,pub_date
0,WRxqtfcfkmbd,"""'A Man of Letters, a Man of Business:' Edwar...",Trudi Abel,[0136737-1242970-8],,,1993-01-01
1,ddccW7_MepAn,"""Extraoperative"" MRI (eoMRI) for Brain Tumor S...",Muhammad Abd-El-Barr,[0791319-1270473-8],BACKGROUND: There is accumulating evidence tha...,10.1016/j.wneu.2015.02.002,2015-06-01
2,uQlqL1owm4Bz,"""I'm Leaving"": Factors That Impact Against Med...",Suresh Agarwal,[0812630-1434865-8],BACKGROUND: Discharge against medical advice (...,10.1016/j.jemermed.2019.12.023,2020-04-01
3,5yemfMqfe2ih,"""The eye sees only what the mind is prepared t...",Muhammad Abd-El-Barr,[0791319-1465699-8],BACKGROUND: Intraoperative computed tomography...,10.1016/j.clinimag.2020.11.034,2021-04-01
4,J6dTj1veitNU,'Unresectable' pancreatic cancer: Conceptual c...,James Abbruzzese,[0634557-1110514-8],,,2007-11-01


In [28]:
## create coauthorship table 
# step one (create a table one row per pub_id + researcher pairs (not dedepulicated) 
from itertools import combinations
import pandas as pd

author_pub = raw_pub_df[[
    "pub_title",
    "researcher_id",
    "full_name"
]].dropna(subset=["pub_title", "researcher_id"])

author_pub = author_pub.merge(
    publication_clean_df[["pub_unique_id", "pub_title"]],
    on="pub_title",
    how="left"
)

print("Total author–pub rows:", len(author_pub))
print("Unique publication_unique_id:", author_pub["pub_unique_id"].nunique())

pairs = []

for uid, group in author_pub.groupby("pub_unique_id"):
    authors = group.to_dict("records")

    if len(authors) < 2:
        continue

    for a, b in combinations(authors, 2):

        if a["researcher_id"] < b["researcher_id"]:
            left, right = a, b
        else:
            left, right = b, a

        if left["researcher_id"] == right["researcher_id"]:
            continue

        pairs.append({
            "pub_unique_id": uid,
            "researcher_A_id": left["researcher_id"],
            "researcher_B_id": right["researcher_id"],
            "researcher_A_name": left["full_name"],
            "researcher_B_name": right["full_name"]
        })

coauthor_pairs = pd.DataFrame(pairs)

print("Total raw coauthor pairs:", len(coauthor_pairs))
coauthor_pairs.head()


Total author–pub rows: 389644
Unique publication_unique_id: 3270
Total raw coauthor pairs: 23980158


,pub_unique_id,researcher_A_id,researcher_B_id,researcher_A_name,researcher_B_name
0,-0HhA1hX1Fox,hoxMQY2SKrOS,yJ_jNlX9SMgh,James Abbruzzese,James Abbruzzese
1,-0HhA1hX1Fox,8dnszJhZk_wX,hoxMQY2SKrOS,James Abbruzzese,James Abbruzzese
2,-0HhA1hX1Fox,hoxMQY2SKrOS,s2BpCeJpNTF7,James Abbruzzese,James Abbruzzese
3,-0HhA1hX1Fox,E5iTT8048sf-,hoxMQY2SKrOS,James Abbruzzese,James Abbruzzese
4,-0HhA1hX1Fox,6NmwTtLHh8gM,hoxMQY2SKrOS,James Abbruzzese,James Abbruzzese


In [29]:
coauthor_pairs.head()

,pub_unique_id,researcher_A_id,researcher_B_id,researcher_A_name,researcher_B_name
0,-0HhA1hX1Fox,hoxMQY2SKrOS,yJ_jNlX9SMgh,James Abbruzzese,James Abbruzzese
1,-0HhA1hX1Fox,8dnszJhZk_wX,hoxMQY2SKrOS,James Abbruzzese,James Abbruzzese
2,-0HhA1hX1Fox,hoxMQY2SKrOS,s2BpCeJpNTF7,James Abbruzzese,James Abbruzzese
3,-0HhA1hX1Fox,E5iTT8048sf-,hoxMQY2SKrOS,James Abbruzzese,James Abbruzzese
4,-0HhA1hX1Fox,6NmwTtLHh8gM,hoxMQY2SKrOS,James Abbruzzese,James Abbruzzese


In [30]:
## create coauthorship table
# step two (created the deduplicated pairs with their joint pub counts)
coauthor_pairs_final = (
    coauthor_pairs
        .groupby(
            ["researcher_A_id", "researcher_B_id"],
            as_index=False
        )
        .agg({
            "pub_unique_id": "nunique",          
            "researcher_A_name": "first",        
            "researcher_B_name": "first"
        })
        .rename(columns={"pub_unique_id": "joint_publications"})
)



In [31]:
coauthor_pairs_final.head(100)

,researcher_A_id,researcher_B_id,joint_publications,researcher_A_name,researcher_B_name
0,--4Hlb_HNyki,-2R24mY47U20,191,Suresh Agarwal,Suresh Agarwal
1,--4Hlb_HNyki,-do_LXWBU-u4,191,Suresh Agarwal,Suresh Agarwal
2,--4Hlb_HNyki,02GcezgjFpm5,191,Suresh Agarwal,Suresh Agarwal
3,--4Hlb_HNyki,0AAhgRFBF-fE,191,Suresh Agarwal,Suresh Agarwal
4,--4Hlb_HNyki,0By2sV1nnzcW,191,Suresh Agarwal,Suresh Agarwal
...,...,...,...,...,...
95,--4Hlb_HNyki,mtNQVJ8TIZUU,191,Suresh Agarwal,Suresh Agarwal
96,--4Hlb_HNyki,pC_9kJBlXGQY,191,Suresh Agarwal,Suresh Agarwal
97,--4Hlb_HNyki,pcXyOHMEOT0d,191,Suresh Agarwal,Suresh Agarwal
98,--4Hlb_HNyki,qDFy_OlAMpSz,191,Suresh Agarwal,Suresh Agarwal


In [32]:
researcher_df[researcher_df["full_name"] == "Alfred Goshaw"][["researcher_id", "full_name", "pub_count"]]


,researcher_id,full_name,pub_count


In [33]:
import pandas as pd

# 1️⃣ 在 researcher_df 里找发表数 > 1000 的作者
high_pub_authors = researcher_df[researcher_df["pub_count"] > 1000][
    ["researcher_id", "full_name", "pub_count"]
]

print("Researchers with pub_count > 1000:")
print(high_pub_authors)

# 只挑前 3 个作者做检查（你可以改成 5、10 等）
check_authors = high_pub_authors.head(3)

for _, row in check_authors.iterrows():
    rid = row["researcher_id"]
    name = row["full_name"]
    reported_count = row["pub_count"]
    
    # 在 raw_pub_df 里查这个作者对应的所有记录
    author_pubs = raw_pub_df[raw_pub_df["researcher_id"] == rid]
    
    # 逐行计数（接口返回的“原始条数”）
    total_rows = len(author_pubs)
    
    # 按标题去重后的真正“不同论文数”
    unique_titles = author_pubs["pub_title"].nunique()
    
    print("\nAuthor:", name)
    print("  researcher_df pub_count:       ", reported_count)
    print("  raw_pub_df total rows:         ", total_rows)
    print("  raw_pub_df unique pub_title:   ", unique_titles)


Researchers with pub_count > 1000:
Empty DataFrame
Columns: [researcher_id, full_name, pub_count]
Index: []
